## User Management (Keycloak)
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import utils.DemoKeycloakClient as Client
import jwt
import json

base_domain = "develop.eoepca.org"
platform_domain = f"keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"

## Client
We instantiate an object to interact with the platform. The object dynamically registers two Keycloak clients, one with admin privileges, another to act as a resource server to demo resources protection using UMA (User Managed Access).

In [ ]:
keycloak = Client.DemoKeycloakClient(server_url, realm, "admin", "admin_Abcd1234#")

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

#### Create Users
Create two users, an Eric with user and user-premium roles, and an Alice with just user role.

In [ ]:
eric_id = keycloak.create_user("eric", "eric", ["user", "user-premium"])
alice_id = keycloak.create_user("alice", "alice", ["user"])

#### Authentication

In [ ]:
token = keycloak.get_user_token("eric", "eric")
print("Eric token:\n" + json.dumps(token, indent = 2))

#### Inspect Eric User Token

In [ ]:
access_token = token['access_token']
jwt_header = jwt.get_unverified_header(access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

#### Inspect Alice User Token

In [ ]:
access_token = token['access_token']
jwt_header = jwt.get_unverified_header(access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Protect resource
Access a protected resource using UMA flow

#### Assign roles to users
Roles are used to define policies that will protect resources based on roles - Role-based access control (RBAC)

In [ ]:
keycloak.create_realm_role('user')
keycloak.create_realm_role('user-premium')

keycloak.assign_realm_role_to_user(eric_id, 'user')
keycloak.assign_realm_role_to_user(eric_id, 'user-premium')

keycloak.assign_realm_role_to_user(alice_id, 'user')

#### Register resources
Register public and protected resources

In [ ]:
keycloak.register_resources()

Right now, resources are protected by a default policy, which grants access to users within the realm.
Let's see how Keycloak protects resources using role based policies.

#### Register policies
Register role based policies

In [ ]:
keycloak.register_policies()

#### Register resource permissions
Resources permissions are set by assigning policies to resources

In [ ]:
keycloak.assign_resource_permissions()

#### Verify access to

#### Get PAT (Personal Access Token)
Personal Access Token is used to access Keycloak's Protection API, which manages resources and policies.

In [ ]:
pat = keycloak.generate_pat()
print(pat)

### Get Resource ID

In [ ]:
resource_id = keycloak.get_resource_id_from_name("Premium Resource")[0]